# Ryan's First Model - Geospatial ML "Hello World"
## Prepare to have your mind blown!
10/4/2023 rhprasad@outlook.com

So we got 80k labeled continent points holed up in PostGIS on RDS, yearning for a more meaningful existence. Can you hear their cries? Let's unleash these puppies upon the world! And by the world... I mean scikit-learn.

What's my IP? I need this because I will need to whitelist it if it has changed since my last experiment connecting to RDS.

In [3]:
from requests import get
ip = get('https://api.ipify.org').content.decode ('utf8') 
print ('My public IP address is: {}'.format (ip))

My public IP address is: 52.27.224.182


Yep, it has changed. Whatever. I am sure there is a good reason for this and I am not using SageMaker Studio as the creators intended. I'll just go change that in the security group settings for my VPC. Some dude on YouTube was saying the only way to get data in is via S3... he may be right! I am very new to this.

![](images/inbound_rules_1004.png)

Okay let's test that out!

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import psycopg2
import pandas as pd

engine = create_engine(
    "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}".format(
        dialect="postgresql",
        driver="psycopg2",
        username="postgres",
        password="postgres",
        host="fastdb.com40arouubf.us-west-2.rds.amazonaws.com", 
        port=5432,
        database="geoml"
    )
)

with engine.connect() as db_conn:
    sql_query = "SELECT version()"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,version
0,"PostgreSQL 15.3 on x86_64-pc-linux-gnu, compil..."


Looks like psycopg2 (db driver that allows us to connect to PostgreSQL) needs to be reinstalled every time we summon it, since these instances are not persistent. If you were wondering, it is easy to do this inline:

`%pip install psycopg2-binary`

Regardless, looks like we are connected to PostGIS and are ready to rock and roll! Last time, I forgot to check the coordinate system of the data and this is the first item on the to-do list.

In [9]:
with engine.connect() as db_conn:
    sql_query = "SELECT ST_SRID (geom) FROM points_xy LIMIT 1;"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,st_srid
0,4326


According to [this document](https://developers.arcgis.com/rest/services-reference/enterprise/pdf/gcs_pdf_11.1.pdf), this Well-Known ID corresponds to WGS84. So all seems to be in order. Let's examine the data.

In [10]:
with engine.connect() as db_conn:
    sql_query = "SELECT * from points_xy LIMIT 10;"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,id,geom,cid
0,1,0101000020E6100000E0B4617AE6982D40C0D9B3AF8860...,1
1,2,0101000020E6100000402327D98BEF3E400024E42D9E84...,1
2,3,0101000020E6100000F0D4F6F8827C3E40002B89D02E6F...,1
3,4,0101000020E6100000E08804F35A392D40308F9C75DB4F...,1
4,5,0101000020E610000060DB7D9680EA314090BDEB012D61...,1
5,6,0101000020E6100000A0CE97B6B6D03B4020F190226522...,1
6,7,0101000020E610000000008E17F45D404040B9E9E81B3B...,1
7,8,0101000020E610000000475410520B13C040BE458003CC...,1
8,9,0101000020E610000080AB9CDC402C0FC0780C37696528...,1
9,10,0101000020E6100000881214493833424080CE3CFDC342...,1


Looks like we need to get the geometry into longitude / latitude features in Pandas. Then we will need to shuffle the data, since it looks like ArcGIS created the dataset one continent at a time. Let's get rid of that nasty hex.

In [11]:
with engine.connect() as db_conn:
    sql_query = "SELECT id, ST_AsText(geom), cid from points_xy LIMIT 10;"
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,id,st_astext,cid
0,1,POINT(14.798633408000057 10.688542834000032),1
1,2,POINT(30.93572766500006 -0.797438706999969),1
2,3,POINT(30.486373482000033 -15.717154041999947),1
3,4,POINT(14.612022013000058 -16.311942434999935),1
4,5,POINT(17.916024595000067 22.379593010000065),1
5,6,POINT(27.815287983000076 8.567177849000075),1
6,7,POINT(32.73401159700006 5.057723655000075),1
7,8,POINT(-4.761055235999947 12.898464211000032),1
8,9,POINT(-3.896608088999926 32.31559481800008),1
9,10,POINT(36.40015519600007 -14.630401528999982),1


Now... how to get this text into two floating point columns in pandas... looks like PostGIS has us covered yet again. Amazing software.

In [13]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT ST_X (ST_Transform (geom, 4326)) AS lon,
       ST_Y (ST_Transform (geom, 4326)) AS lat,
	   cid as continent_id
    FROM points_xy LIMIT 10;
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df

,lon,lat,continent_id
0,14.798633,10.688543,1
1,30.935728,-0.797439,1
2,30.486373,-15.717154,1
3,14.612022,-16.311942,1
4,17.916025,22.379593,1
5,27.815288,8.567178,1
6,32.734012,5.057724,1
7,-4.761055,12.898464,1
8,-3.896608,32.315595,1
9,36.400155,-14.630402,1


Now to shuffle these guys. So far we have only seen 10 points in Africa, since 1 is the continent ID for Africa.

In [19]:
with engine.connect() as db_conn:
    sql_query = """
    SELECT ST_X (ST_Transform (geom, 4326)) AS lon,
       ST_Y (ST_Transform (geom, 4326)) AS lat,
	   cid as continent_id
    FROM points_xy;
    """
    df = pd.read_sql_query(sql=text(sql_query), con=db_conn)
df = df.sample(frac=1)
df

,lon,lat,continent_id
59821,176.739902,-38.549811,6
46569,-92.860529,15.353856,5
30701,9.583069,48.472944,4
10178,115.125888,37.617761,2
978,5.981260,22.637008,1
...,...,...,...
17996,96.940243,39.420517,2
53893,167.279505,-21.080929,6
65810,-43.570083,-18.181387,7
4586,35.489182,21.725386,1


Oookay. Lets get lon and lat into a NumPy array as the two features for the model, and continent_id into another array as the target for scikit-learn. For this exercise, I decided against testing the model here since I am dying to get this thing deployed and hooked up to the front end. Thus, I am not doing a test split. Supposedly I can swap out models pretty easily with the MLops tools in SageMaker and will play with that later. 

In [22]:
features = df[["lon", "lat"]].to_numpy()

In [23]:
features

array([[176.73990212, -38.5498108 ],
       [-92.86052899,  15.35385642],
       [  9.58306884,  48.47294361],
       ...,
       [-43.57008325, -18.18138722],
       [ 35.48918157,  21.72538618],
       [ 43.40116735, -87.21958985]])

In [34]:
target = df[["continent_id"]].to_numpy()

Well this was relatively painless. Looks like everything lines up and this path is well trodden. I was expecting a struggle... anyway let's get scikit-learn fired up. 

**I am going to admit here that I do not know how this algorithm in scikit-learn works! I just want this thing to hook up to my front end!!** I just like programming and maps, okay?

In [39]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC(verbose=0, max_iter=10000)
print(lsvc)

LinearSVC(max_iter=10000)


In [ ]:
lsvc.fit(features, target.ravel())
score = lsvc.score(features, target.ravel())
print("Score: ", score)

Score:  0.6872


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


My professor mentioned that I should probably steer clear of the linear classifiers on unprojected data if accuracy is important. But this is not Kaggle and of course, that's the first one I chose! For science!

Yeah, I really, really want to fiddle with this now. There is a lot of things to mess with - but holding off for now. It's just a warning and I don't really care if it converges. Actually, I do, but my front end is calling. 

Anyway **IT'S ALIVE MUAHAHA**

In [41]:
lsvc.predict([[16,8]])

array([1])

In [42]:
lsvc.predict([[-98,36]])

array([5])

![](images/points_ids_nonb.png)

Yep I am already happy with this. That was two consective correct predictions for Africa and North America. Gonna get some funky results in the water, but I'll play around with that later. Next up... deployment! 